In [ ]:
import rospy
from door_opener.srv import HandleDetection
from visualization_msgs.msg import Marker
from ros_tensorflow_msgs.srv import *
from sensor_msgs.msg import PointCloud, PointCloud2, PointField
from sensor_msgs import point_cloud2 as pc2
from manipulation_test.srv import *
import tf2_ros
import numpy as np
import tf.transformations as tf_trans
from geometry_msgs.msg import Pose, Point, Quaternion, PoseStamped

import moveit_commander
import moveit_msgs.msg
from moveit_msgs.msg import RobotState
from sensor_msgs.msg import JointState
from moveit_msgs.msg import AttachedCollisionObject
from moveit_msgs.msg import CollisionObject

import actionlib
from control_msgs.msg import (
    FollowJointTrajectoryAction,
    FollowJointTrajectoryGoal,
    GripperCommandAction,
    GripperCommandGoal,
)

In [ ]:
def convert_pointcloud_to_pointcloud2(pc):
    '''
    Converts a ROS PointCloud message to a ROS PointCloud2 message.
    
    :param pc: a ROS PointCloud message
    :type pc: PointCloud
    :returns: a ROS PointCloud2 message
    :rtype: PointCloud2
    '''

    # Create a list of 3-tuples where each tuple is (x, y, z)
    pc2_data = [(p.x, p.y, p.z) for p in pc.points]

    # Create header
    header = pc.header

    # Create a PointCloud2 message
    pc2_msg = pc2.create_cloud_xyz32(header, pc2_data)

    return pc2_msg

def pointcloud_to_numpy(point_cloud):
    points = []

    for point in point_cloud.points:
        points.append([point.x, point.y, point.z])

    return np.array(points)

def numpy_to_pointcloud2(points):
    """
    Converts a numpy array to a sensor_msgs/PointCloud2 message.

    :param points: Numpy array of point cloud
    :type points: numpy.ndarray
    :returns: sensor_msgs/PointCloud2 message
    :rtype: PointCloud2
    """
    # Create a list of PointField
    fields = [PointField('x', 0, PointField.FLOAT32, 1),
              PointField('y', 4, PointField.FLOAT32, 1),
              PointField('z', 8, PointField.FLOAT32, 1)]

    # Create a header
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = 'base_link'  # replace with your frame id

    # Convert the points array to PointCloud2
    point_cloud2 = pc2.create_cloud(header, fields, points)

    return point_cloud2

def get_rotation_matrix(direction):
    """
    Computes the rotation matrix that will align the given direction vector with the up vector [0, 0, 1].

    :param direction: 3D direction vector
    :type direction: numpy.ndarray
    :return: 3x3 rotation matrix
    :rtype: numpy.ndarray
    """
    # Normalize the direction vector
    direction = direction / np.linalg.norm(direction)

    # Define the up vector
    up = np.array([0, 0, 1])

    # Compute the rotation axis via the cross product
    axis = np.cross(direction, up)
    axis = axis / np.linalg.norm(axis)

    # Compute the rotation angle via the dot product
    angle = np.arccos(np.dot(direction, up))

    # Compute the rotation matrix using the axis-angle formula
    K = np.array([[0, -axis[2], axis[1]],
                  [axis[2], 0, -axis[0]],
                  [-axis[1], axis[0], 0]])

    rotation_matrix = np.eye(4)
    rotation_matrix[:3, :3] = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)
    return rotation_matrix

def pose_to_matrix(pose):
    """
    Converts a geometry_msgs/Pose message to a 4x4 transformation matrix.

    :param pose: A geometry_msgs/Pose message
    :type pose: Pose
    :return: 4x4 transformation matrix
    :rtype: numpy.ndarray
    """
    # Extract the translation
    translation = np.array([pose.position.x, pose.position.y, pose.position.z])

    # Extract the rotation (as a quaternion)
    quaternion = np.array([pose.orientation.x, pose.orientation.y, pose.orientation.z, pose.orientation.w])

    # Convert the quaternion to a 3x3 rotation matrix
    rotation_matrix = tf_trans.quaternion_matrix(quaternion)

    # Create a 4x4 transformation matrix
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    return transformation_matrix

def matrix_to_pose(matrix):
    """
    Converts a 4x4 transformation matrix to a geometry_msgs/Pose message.

    :param matrix: 4x4 transformation matrix
    :type matrix: numpy.ndarray
    :return: A geometry_msgs/Pose message
    :rtype: Pose
    """
    # Extract the translation
    translation = matrix[:3, 3]

    #     # Extract the rotation (as a 3x3 matrix)
    #     rotation_matrix = matrix[:3, :3]

    # Convert the rotation matrix to a quaternion
    quaternion = tf_trans.quaternion_from_matrix(matrix)

    # Create a Pose message
    pose = Pose()
    pose.position = Point(*translation)
    pose.orientation = Quaternion(*quaternion)

    return pose

def pose_on_plane_close_to_origin(plane):
    """
    Computes a pose on the plane, with the position being the point on the plane closest to the origin.

    :param plane: The coefficients [A, B, C, D] of the plane equation Ax + By + Cz + D = 0
    :type plane: numpy.ndarray
    :return: The pose on the plane
    :rtype: Pose
    """
    # Extract the normal vector and distance from the plane parameters
    normal = plane[:3]
    d = plane[3]

    # Normalize the normal vector
    normal = normal / np.linalg.norm(normal)

    # Choose the point on the plane closest to the origin for the position
    position = -d / np.dot(normal, normal) * normal

    # Choose a vector orthogonal to the normal vector as the X-axis
    if normal[0] != 0:
        x_axis = np.array([normal[1], -normal[0], 0])
    elif normal[1] != 0:
        x_axis = np.array([-normal[1], normal[0], 0])
    else:  # normal[2] != 0
        x_axis = np.array([0, -normal[2], normal[1]])

    # Normalize the X-axis vector
    x_axis = x_axis / np.linalg.norm(x_axis)

    # Compute the Y-axis vector as the cross product of the normal and X-axis vectors
    y_axis = np.cross(normal, x_axis)

    # Construct a rotation matrix from the axes
    rotation_matrix = np.eye(4)
    rotation_matrix[:3, :3] = np.vstack((x_axis, y_axis, normal)).T

    # Convert the rotation matrix to a quaternion
    orientation = tf_trans.quaternion_from_matrix(rotation_matrix)

    # Return the pose
    return Pose(position=Point(*position), orientation=Quaternion(*orientation))

#open gripper
def open_gripper(gripper_client):
    goal = GripperCommandGoal()
    goal.command.position = float(0.08)
    goal.command.max_effort = 100
    gripper_client.send_goal_and_wait(goal)

# close gripper
def close_gripper(gripper_client):
    goal = GripperCommandGoal()
    goal.command.position = float(0.0)
    goal.command.max_effort = 100
    gripper_client.send_goal_and_wait(goal)

In [ ]:
rospy.init_node('handle_detection_client')

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
move_group = moveit_commander.MoveGroupCommander("arm") 
scene = moveit_commander.PlanningSceneInterface()
# move_group.set_max_velocity_scaling_factor(1.0)

gripper_client = actionlib.SimpleActionClient(
                "/gripper_controller/gripper_action", GripperCommandAction
            )
gripper_client.wait_for_server()

In [ ]:
rospy.wait_for_service('handle_detection')

In [ ]:
# get camera transform from tf tree
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

In [ ]:
try:
    camera_trans = tfBuffer.lookup_transform('base_link', 'head_camera_rgb_optical_frame', rospy.Time())
except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
    print("tf error")

In [ ]:
# Create a service proxy
handle_detection_service = rospy.ServiceProxy('handle_detection', HandleDetection)

In [ ]:
res = handle_detection_service(camera_trans)
print 'get ', len(res.handle_motions), ' handles from camera'

In [ ]:
# add table into the planning scene
drawer_pose_stamped = geometry_msgs.msg.PoseStamped()
drawer_pose_stamped.header.frame_id = "base_link"
# drawer_pose_stamped.pose = pose_on_plane_close_to_origin(coef_test)
drawer_pose_stamped.pose = pose_on_plane_close_to_origin(np.array(res.handle_motions[0].drawer_plane.coef))
drawer_name = "drawer"
scene.add_box(drawer_name, drawer_pose_stamped, size=(2.0, 2.0, 0.005))

In [ ]:
# marker_pub = rospy.Publisher("/visualization_marker", Marker, queue_size = 2)

In [ ]:
# marker_list = []
# marker_id_temp = 0
# for handle_motion in res.handle_motions:
#     marker = Marker()

#     marker.header.frame_id = "base_link"
#     marker.header.stamp = rospy.Time.now()

#     # set shape, Arrow: 0; Cube: 1 ; Sphere: 2 ; Cylinder: 3
#     marker.type = 2
#     marker.id = marker_id_temp

#     # Set the scale of the marker
#     marker.scale.x = 0.05
#     marker.scale.y = 0.05
#     marker.scale.z = 0.05

#     # Set the color
#     marker.color.r = 0.0
#     marker.color.g = 1.0
#     marker.color.b = 0.0
#     marker.color.a = 1.0

#     # Set the pose of the marker
#     marker.pose.position.x = handle_motion.handle_direction[0]
#     marker.pose.position.y = handle_motion.handle_direction[1]
#     marker.pose.position.z = handle_motion.handle_direction[2]
#     marker.pose.orientation.x = 0.0
#     marker.pose.orientation.y = 0.0
#     marker.pose.orientation.z = 0.0
#     marker.pose.orientation.w = 1.0
    
#     marker_list.append(marker)
#     marker_id_temp += 1
    
#     arrow_marker = Marker()
#     arrow_marker.header.frame_id = "base_link"
#     arrow_marker.header.stamp = rospy.Time.now()

#     # set shape, Arrow: 0; Cube: 1 ; Sphere: 2 ; Cylinder: 3
#     arrow_marker.type = 0
#     arrow_marker.id = marker_id_temp

#     # Set the scale of the marker
#     arrow_marker.scale.x = 0.01
#     arrow_marker.scale.y = 0.01
#     arrow_marker.scale.z = 0.01

#     # Set the color
#     arrow_marker.color.r = 1.0
#     arrow_marker.color.g = 0.0
#     arrow_marker.color.b = 0.0
#     arrow_marker.color.a = 1.0
    
#     start = Point(handle_motion.handle_direction[0], handle_motion.handle_direction[1], handle_motion.handle_direction[2])
#     end = Point(handle_motion.handle_direction[0] + handle_motion.handle_direction[3] * 0.3, 
#                 handle_motion.handle_direction[1] + handle_motion.handle_direction[4] * 0.3, 
#                 handle_motion.handle_direction[2] + handle_motion.handle_direction[5] * 0.3)

#     arrow_marker.points = [start, end]
    
#     marker_list.append(arrow_marker)
#     marker_id_temp += 1

In [ ]:
# # publish the marker for visualization
# while not rospy.is_shutdown():
#     for m in marker_list:
#         marker_pub.publish(m)
#     rospy.rostime.wallsleep(1.0)

In [ ]:
rospy.wait_for_service('grasp_predict')

In [ ]:
grasp_predictor = rospy.ServiceProxy('grasp_predict', Predict)

In [ ]:
# at this point, we select the first handle to open the drawer.
handle_pointcloud_numpy = pointcloud_to_numpy(res.handle_motions[0].handle_pc)

plane_numpy = np.array(res.handle_motions[0].drawer_plane.coef)
open_rotate_mat = get_rotation_matrix(np.array([res.handle_motions[0].handle_direction[3], 
                                           res.handle_motions[0].handle_direction[4], 
                                           res.handle_motions[0].handle_direction[5]]))
rotated_handle_pointcloud_numpy = np.dot(open_rotate_mat, 
                                         np.hstack((handle_pointcloud_numpy, 
                                                    np.ones((handle_pointcloud_numpy.shape[0], 1)))).T).T[:, :3]
try:
    predicted_grasp_result = grasp_predictor(numpy_to_pointcloud2(rotated_handle_pointcloud_numpy), 
                                             numpy_to_pointcloud2(rotated_handle_pointcloud_numpy), 
                                             camera_trans)
except rospy.ServiceException as e:
    print("Service call failed: %s"%e)
    
filtered_grasp_poses_for_vis = []
filtered_grasp_poses = []


for i in range(len(predicted_grasp_result.predicted_grasp_poses)):
    grasp_pose_mat = pose_to_matrix(predicted_grasp_result.predicted_grasp_poses[i].pose)
    # need to filter the grasp pose which may hit the drawer.
    collision_detect_mat = np.dot(np.linalg.inv(open_rotate_mat), grasp_pose_mat)
    
    finger1_collision_detect_mat = np.dot(collision_detect_mat, np.array([0.12,0.4,0,1]))
    finger2_collision_detect_mat = np.dot(collision_detect_mat, np.array([0.12,-0.4,0,1]))
    
    c1 = np.dot(plane_numpy[:3], collision_detect_mat[:3, 3]) + plane_numpy[3] > 0
    c2 = np.dot(plane_numpy[:3], finger1_collision_detect_mat[:3]) + plane_numpy[3] > 0
    c3 = np.dot(plane_numpy[:3], finger2_collision_detect_mat[:3]) + plane_numpy[3] > 0
    
    if (c1 and c2 and c3) or (not (c1 or c2 or c3)):
        grasp_pose_stamped = PoseStamped()
        grasp_pose_stamped.header.stamp = rospy.Time.now()
        filtered_grasp_pose_mat = np.dot(np.linalg.inv(open_rotate_mat), grasp_pose_mat)
        filtered_grasp_poses.append(filtered_grasp_pose_mat)
        grasp_pose_stamped.pose= matrix_to_pose(filtered_grasp_pose_mat)
        filtered_grasp_poses_for_vis.append(grasp_pose_stamped)

In [ ]:
rospy.wait_for_service('visualize_regrasp')
grasp_visualizer = rospy.ServiceProxy('visualize_regrasp', VisualizeRegrasp)
grasp_visualizer(filtered_grasp_poses_for_vis, 
                 [0.08 for _ in range(len(filtered_grasp_poses_for_vis))], 
                 [0 for _ in range(len(filtered_grasp_poses_for_vis))])

In [ ]:
import random
from ros_numpy import numpify, msgify
import tf

In [ ]:
display_trajectory_publisher = rospy.Publisher(
            "/move_group/display_planned_path",
            moveit_msgs.msg.DisplayTrajectory,
            queue_size=10,
        )

In [ ]:
# open gripper
open_gripper(gripper_client)

In [ ]:
grasp_shift = np.array([[1,0,0,0.02],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
pre_grasp_shift = np.array([[1,0,0,-0.1],[0,1,0,0],[0,0,1,0],[0,0,0,1]])

pre_grasp_pose = np.eye(4)
grasp_pose = np.eye(4)
open_drawer_pose = np.eye(4)
# need to plan the motion to pre-grasp pose.
random.shuffle(filtered_grasp_poses)
for g in filtered_grasp_poses[:5]:
    move_group.set_start_state_to_current_state()
    # generate grasp pose
    grasp_pose = g.dot(grasp_shift)

    # calculate the pre-grasp pose
    pre_grasp_pose = grasp_pose.dot(pre_grasp_shift)
    
    # pose to move it for opening the drawer
    open_drawer_pose = grasp_pose.copy()
    open_drawer_pose[0,3] += res.handle_motions[0].handle_direction[3] * 0.1
    open_drawer_pose[1,3] += res.handle_motions[0].handle_direction[4] * 0.1
    open_drawer_pose[2,3] += res.handle_motions[0].handle_direction[5] * 0.1
    
    trans = tf.transformations.translation_from_matrix(pre_grasp_pose).tolist()
    quat = tf.transformations.quaternion_from_matrix(pre_grasp_pose).tolist()

    move_group.clear_pose_targets()
    move_group.set_pose_target(trans + quat)
    plan_result = move_group.plan()
    
    if plan_result[0]:
        break

In [ ]:
display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = move_group.get_current_state()
display_trajectory.trajectory.append(plan_result[1])
# display_trajectory.trajectory.append(opening_plan)
# Publish
display_trajectory_publisher.publish(display_trajectory)

In [ ]:
move_group.execute(plan_result[1])

In [ ]:
move_group.set_start_state_to_current_state()
(approach_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, grasp_pose)], 0.01, 0.0)
print fraction
# check whether you can place the object
if fraction > 0.9:
    print "good to go"

In [ ]:
display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = move_group.get_current_state()
display_trajectory.trajectory.append(approach_plan)
# Publish
display_trajectory_publisher.publish(display_trajectory)

In [ ]:
move_group.execute(approach_plan)

In [ ]:
# close gripper
close_gripper(gripper_client)

In [ ]:
move_group.set_start_state_to_current_state()
(opening_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, open_drawer_pose)], 0.01, 0.0)
print fraction
# check whether you can place the object
if fraction > 0.9:
    print "good to go"

In [ ]:
display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = move_group.get_current_state()
display_trajectory.trajectory.append(opening_plan)
# Publish
display_trajectory_publisher.publish(display_trajectory)

In [ ]:
move_group.execute(opening_plan)

In [ ]:
# open gripper
open_gripper(gripper_client)